# Robsut Text Normalisation

This notebook is used to apply robust normalisation to preprocessed text data before textual analysis to improve model accuracy and reduce computational time.

Entities will be removed from text are: 
- PERSON: Names of individuals.
- ORG: Names of organizations, including companies, governmental entities, and other groups.
- NORP: Nationalities, religious and political groups.
- FAC: Facilities, like buildings, airports, highways, bridges.
- GPE: Geo-political entities, such as countries, cities, states.
- LOC: Non-GPE locations, mountain ranges, bodies of water.
- PRODUCT: Objects, vehicles, foods, etc. (not services).
- EVENT: Named events, such as battles, wars, sports events, hurricanes, etc.
- WORK_OF_ART: Titles of books, songs, and other works of art.
- LAW: Named documents made into laws, including directives, regulations, and legislative acts.
- LANGUAGE: Any named language.
- DATE: Absolute or relative dates or periods.
- TIME: Times smaller than a day.
- PERCENT: Percentage (including "%").
- MONEY: Monetary values, including unit.
- QUANTITY: Measurements, as of weight or distance.
- ORDINAL: "first", "second", etc.
- CARDINAL: Numerals that do not fall under another type (e.g., a counting number).

We relied on spacy `EntityRecognizer` to identify the entities in the text. Due to our computational power constraints, the small model of spaCy web English which is `en_core_web_sm` was used instead of `en_core_web_trf` to trade some accuracy for speed and efficiency.

Output of this notebook: `review_camera_normalised.csv`, `review_phone_normalised.csv`,`review_computer_normalised.csv` and `review_subset_normalised.csv`

## Required Libraries

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_sm")

In [2]:
ent_labels_to_remove = ["PERSON", "ORG", "NORP", "FAC", "GPE", "LOC", "PRODUCT", 
                          "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE", "DATE", "TIME", 
                          "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"] # all possible entity in spacy entity recognition

## Define different normalisation functions

In [3]:
# normalise a text string, use with .apply()

def normalise_text(text):
    text = text.replace("\n", " ")
    text = text.lower()
    doc = nlp(text)
    tokens = []
    ents_to_exclude_index = set()

    for ent in doc.ents:
        if ent.label_ in ent_labels_to_remove:
            ents_to_exclude_index.update(range(ent.start, ent.end))

    for token in doc:
        if (
            not token.like_url
            and not token.like_email
            and not token.is_stop
            and not token.is_punct
            and token.is_alpha
            and token.i not in ents_to_exclude_index):
                tokens.append(token.lemma_.lower())
    return " ".join(tokens)  # return a string
    # return tokens # return a list


def hash_to_word(test_text):
    word = [nlp.vocab.strings[hash] for hash in test_text]
    return word

In [4]:
# normalise a column in list form consists of spacy nlp object
# this method is the fastest but consume huge memory at once

def normalise_doc_at_once(nlp_object_column_list, ent_labels_to_remove):
    results = []
    for doc in tqdm(nlp_object_column_list):
        tokens = []
        for token in doc:
            if (
                not token.like_url
                and not token.like_email
                and not token.is_stop
                and not token.is_punct
                and token.is_alpha
                and token.ent_type_ not in ent_labels_to_remove
            ):
                tokens.append(token.lemma_.lower())
        results.append(" ".join(tokens))
    return results

def column_to_nlp_object_list_at_once(a_df_column):
    return list(tqdm(nlp.pipe(a_df_column.tolist())))

In [5]:
#normalise in batches to prevent out of memory error

def normalise_doc(doc, ent_labels_to_remove):
    tokens = [
        token.lemma_.lower() for token in doc
        if not token.like_url
        and not token.like_email
        and not token.is_stop
        and not token.is_punct
        and token.is_alpha
        and token.ent_type_ not in ent_labels_to_remove
    ]
    return " ".join(tokens)

def process_column_in_batches(df_column, batch_size=50000, n_process=1):
    results = []
    for i in tqdm(range(0, len(df_column), batch_size)):
        batch = df_column[i:i+batch_size].tolist()
        docs = nlp.pipe(batch, n_process=n_process)
        for doc in docs:
            normalized_text = normalise_doc(doc, ent_labels_to_remove)
            results.append(normalized_text)
    return results


## Load preprocessed data

In [6]:
df_review = pd.read_csv('review.csv')
# the warning was caused by uncleaned and mixed datatype in vote column, we will deal with this when we need vote data

/var/folders/nq/z_km61px217dlw3_jg7krlvc0000gn/T/ipykernel_41327/3202621294.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_review = pd.read_csv('review.csv')


In [7]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20566364 entries, 0 to 20566363
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   verified    bool   
 2   reviewTime  object 
 3   asin        object 
 4   reviewText  object 
 5   vote        object 
 6   image       bool   
 7   Year        int64  
 8   price       float64
 9   main_cat    object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 1.3+ GB


In [8]:
# make sure there is no NA in reviewText column as this will cause error in spacy nlp.pipe
df_review["reviewText"].isna().sum()

0

In [9]:
# show top 10 main product category
df_review['main_cat'].value_counts().head(10)

Computers                    6732262
All Electronics              3655780
Home Audio & Theater         3301869
Camera & Photo               2502223
Cell Phones & Accessories    2272921
Car Electronics               412355
Amazon Devices                296778
Sports & Outdoors             221047
Tools & Home Improvement      198603
Office Products               172867
Name: main_cat, dtype: int64

## Normalisation

### Normalise Camera & Photo  review

In [17]:
df_review_camera = df_review.loc[df_review["main_cat"] == "Camera & Photo" ]
# df_review_camera.to_csv('review_camera.csv', index=False) # unnormalised version to csv if needed

In [18]:
df_review_camera["reviewText"]

12386       I was skeptical about buying a generic replace...
12387       Battery arrived ahead of schedule and was 1/2 ...
12388       Muy importante tener una batera cargada de rep...
12389       The two rechargeable battery packs I ordered a...
12390       Battery charged quickly and installed in my ca...
                                  ...                        
20566300    I love this dry box!!!! Besides being extremel...
20566309    If you have more than day 5000 dollars of gear...
20566310                                  Highly Recommended.
20566313    I have been using this camera for about 5 mont...
20566314    I enjoyed how durable and small this product i...
Name: reviewText, Length: 2502223, dtype: object

In [20]:
df_review_camera['reviewText'] = process_column_in_batches(df_review_camera['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 50000 per batch)

100%|██████████| 51/51 [3:04:29<00:00, 217.05s/it]  
C:\Users\rzp98\AppData\Local\Temp\ipykernel_27388\3079339649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_camera['reviewText'] = process_column_in_batches(df_review_camera['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 50000 per batch)


In [21]:
df_review_camera['reviewText']

12386       skeptical buy generic replacement battery new ...
12387       battery arrive ahead schedule price anyplace b...
12388       muy importante tener una batera cargada de rep...
12389       rechargeable battery pack order work great cam...
12390       battery charge quickly instal camera easily gr...
                                  ...                        
20566300    love dry box extremely functional allow displa...
20566309    gear worth invest professional storage camera ...
20566310                                     highly recommend
20566313    camera truly k great video quality seriously g...
20566314           enjoy durable small product bulky easy use
Name: reviewText, Length: 2502223, dtype: object

In [22]:
df_review_camera.to_csv('review_camera_normalised.csv', index=False)
del df_review_camera

### Normalise Cell Phones & Accessories review

In [23]:
df_review_phone = df_review.loc[df_review["main_cat"] == "Cell Phones & Accessories" ]
# df_review_phone.to_csv('review_phone.csv', index=False) # unnormalised version to csv if needed

In [24]:
df_review_phone["reviewText"]

1245        Stephanie has spent time filtering out many pr...
1246        For the past two years I've taught math profic...
1247        This book has notes, definitions,and practice ...
1248        The resources in this book are like no other. ...
1249        I am a High School mathematics teacher in the ...
                                  ...                        
20566349    Great product, great customer care. Thanks & w...
20566350    Works great, love the longer cord. As with any...
20566351    Perfect length. Very durable braiding. Works g...
20566352    Ok here is an odd thing that happened to me, I...
20566353                                          Works well.
Name: reviewText, Length: 2272921, dtype: object

In [25]:
df_review_phone['reviewText'] = process_column_in_batches(df_review_phone['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 50000 per batch)

  0%|          | 0/46 [00:00<?, ?it/s]

100%|██████████| 46/46 [2:07:19<00:00, 166.07s/it]  
C:\Users\rzp98\AppData\Local\Temp\ipykernel_27388\1626423066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_phone['reviewText'] = process_column_in_batches(df_review_phone['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 50000 per batch)


In [26]:
df_review_phone['reviewText']

1245        spend time filter problem appear test book lay...
1246        teach math proficiency class high school senio...
1247        book note definition practice problem explaint...
1248        resource book like product student love teach ...
1249        high school mathematic teacher district teach ...
                                  ...                        
20566349     great product great customer care thank business
20566350    work great love long cord cord buy long expect...
20566351    perfect length durable braiding work great rap...
20566352    ok odd thing happen click sale item ft cable r...
20566353                                                 work
Name: reviewText, Length: 2272921, dtype: object

In [27]:
df_review_phone.to_csv('review_phone_normalised.csv', index=False)
del df_review_phone

### Normalise Computer review

In [28]:
df_review_comp = df_review.loc[df_review["main_cat"] == "Computers" ]
# df_review_comp.to_csv('review_computer.csv', index=False) # unnormalised version to csv if needed

In [29]:
df_review_comp["reviewText"]

743         It does 2A and charges a DEAD Nook in a few ho...
744         Same charger can be bought at Barnes & Noble f...
745         Works well, a little pricey I think for a char...
746         My son crewed my HD charger cord so I needed a...
747         It works perfect, puppy chewed last one and I ...
                                  ...                        
20566359    Had it 1 day and it quit working, will be retu...
20566360    Received item in 2 days. Product worked as adv...
20566361    I have it plugged into a usb extension on my g...
20566362              Fast delivery product was simple to use
20566363           Working as advertised, so far no problems.
Name: reviewText, Length: 6732262, dtype: object

In [30]:
df_review_comp['reviewText'] = process_column_in_batches(df_review_comp['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 5000 per batch)

  0%|          | 0/135 [00:00<?, ?it/s]

100%|██████████| 135/135 [6:40:31<00:00, 178.01s/it]  
C:\Users\rzp98\AppData\Local\Temp\ipykernel_27388\4150549989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_comp['reviewText'] = process_column_in_batches(df_review_comp['reviewText'], batch_size=50000) # adjust batch based on memory available (16G memory could handle around 5000 per batch)


In [31]:
df_review_comp['reviewText']

743         charge dead nook heat compare wall wort dual c...
744         charger buy price give understand competition ...
745         work little pricey think charge cable lose ori...
746         son crew hd charger cord need exactly like son...
747         work perfect puppy chew till difference price ...
                                  ...                        
20566359                                     quit work return
20566360    receive item product work advertised instructi...
20566361    plug usb extension gaming pc ideally place tra...
20566362                     fast delivery product simple use
20566363                           work advertise far problem
Name: reviewText, Length: 6732262, dtype: object

In [32]:
df_review_comp.to_csv('review_computer_normalised.csv', index=False)
del df_review_comp

### Normalise subset of review data 

In [16]:
df_review["reviewText"]

0           This was the first time I read Garcia-Aguilera...
1           As with all of Ms. Garcia-Aguilera's books, I ...
2           I've not read any of Ms Aguilera's works befor...
3           This romance novel is right up there with the ...
4           Carolina Garcia Aguilera has done it again.  S...
                                  ...                        
20566359    Had it 1 day and it quit working, will be retu...
20566360    Received item in 2 days. Product worked as adv...
20566361    I have it plugged into a usb extension on my g...
20566362              Fast delivery product was simple to use
20566363           Working as advertised, so far no problems.
Name: reviewText, Length: 20566364, dtype: object

In [17]:
df_review_subset = df_review.sample(frac=0.2, random_state=9) # 1/5 of the entire review dataset = around 4M rows
df_review_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4113273 entries, 6036777 to 303060
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   verified    bool   
 2   reviewTime  object 
 3   asin        object 
 4   reviewText  object 
 5   vote        object 
 6   image       bool   
 7   Year        int64  
 8   price       float64
 9   main_cat    object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 290.3+ MB


In [18]:
df_review_subset["main_cat"].value_counts().head(10)

Computers                    1346617
All Electronics               731408
Home Audio & Theater          659954
Camera & Photo                500418
Cell Phones & Accessories     455020
Car Electronics                82516
Amazon Devices                 59060
Sports & Outdoors              43884
Tools & Home Improvement       39669
Office Products                34832
Name: main_cat, dtype: int64

In [19]:
df_review_subset['reviewText']

6036777     It is a charger that does the job easily and q...
13947242               It's a SDXC card...what's not to like?
1294956     Not a bad product if you are leaning\nthat way...
16563154                               good tab for the price
9143168     It does everything as promised. I can see all ...
                                  ...                        
20261538                Excellet for the shower! Good product
16451992                   case fits kindle voyager perfectly
6733777     a good clear picture very crisp you can really...
9093831     very good, working and very clear strong and t...
303060      I bought this KVM switch for my home office so...
Name: reviewText, Length: 4113273, dtype: object

In [20]:
df_review_subset['reviewText'] = process_column_in_batches(df_review_subset['reviewText'], batch_size=50000, n_process=-1) # adjust batch based on memory available (16G memory could handle around 50000 per batch)

100%|██████████| 83/83 [1:59:56<00:00, 86.71s/it]


In [21]:
df_review_subset['reviewText']

6036777     charger job easily quickly use direction figur...
13947242                                       sdxc card like
1294956     bad product lean way cost reasonable operation...
16563154                                       good tab price
9143168     promise internet goody print network printer p...
                                  ...                        
20261538                         excellet shower good product
16451992                    case fit kindle voyager perfectly
6733777     good clear picture crisp tell s view area get ...
9093831                    good work clear strong lot beating
303060      buy switch home office crawl desk disconnect m...
Name: reviewText, Length: 4113273, dtype: object

In [22]:
df_review_subset.to_csv('review_subset_normalised.csv', index=False)
del df_review_subset